## Exploring the schema

In this notebook we systematically go through the tables' schemas and define the tables/columns which are relevant/useful for modelling and cohort building.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

establish connection to DB and define helper function for running queries

In [5]:
import pandas as pd
from proto.etl.config import SSHInfoEicu, DBInfoEicu
from proto.etl.utils import connect_to_db_via_ssh, run_eicu_query, get_column_completeness, load_schema_for_modelling

conn = connect_to_db_via_ssh(SSHInfoEicu, DBInfoEicu)
cursor = conn.cursor()
query_schema = 'set search_path to eicu_crd;'

### Work out how many patient each hospital has, keep patients from top 5

In [6]:
query = """
    select hospitalid, count(patientunitstayid) as n
    from patient 
    group by hospitalid 
    order by n desc
"""

df_hospitals = run_eicu_query(query, conn)
df_hospitals.head()

,hospitalid,n
0,73,7059
1,167,6092
2,264,5237
3,420,4679
4,176,4328


In [4]:
# 30 hospitals (out of the 208) have half of the patients
df_hospitals.head(30).n.sum()

# for the basic cohort building we'll keep the top 5
top_hospitals = df_hospitals.hospitalid[:5].values

### Execute all SQL sripts from the eicu-code repo

There are a bunch of sql scripts that make pivotted tables with some basic features out of the original messy tables. These scripts were written by the authors of the eicu dataset.

Run those in PyCharm, along with `setup/eicu_sql_scripts/top5hospitals/patient_top5hospitals.sql` which will create a view of the largest 5 hospitals' patients that we can use for the POC.

### Extract schema for all tables

In [35]:
query = """
select table_name from information_schema.tables
where table_schema = 'eicu_crd'
"""
df_tables = run_eicu_query(query, conn)
df_tables

,table_name
0,admissiondrug
1,admissiondx
2,allergy
3,apacheapsvar
4,apachepatientresult
5,apachepredvar
6,careplancareprovider
7,careplaneol
8,careplangeneral
9,careplangoal


In [118]:
df_schemas = None
eicu_tables = df_tables.table_name.values
for table in eicu_tables:
    query = """
    select *
    from information_schema.columns
    where table_schema = 'eicu_crd'
    and table_name =  '%s'
    """ % table
    df_schema = run_eicu_query(query, conn)
    if df_schemas is None:
        df_schemas = df_schema
    else:
        df_schemas = pd.concat([df_schemas, df_schema], axis=0)
df_schemas = df_schemas.reset_index().drop('index', axis=1)
df_schemas.to_csv('../setup/eicu_schema.csv')

In [46]:
df_schemas.tail(25)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
485,eicu,eicu_crd,pivoted_vital,nibp_diastolic,8,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
486,eicu,eicu_crd,pivoted_vital,nibp_mean,9,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
487,eicu,eicu_crd,pivoted_vital,temperature,10,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
488,eicu,eicu_crd,pivoted_vital,temperaturelocation,11,None,YES,text,None,1.07374e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
489,eicu,eicu_crd,pivoted_vital,ibp_systolic,12,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
490,eicu,eicu_crd,pivoted_vital,ibp_diastolic,13,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
491,eicu,eicu_crd,pivoted_vital,ibp_mean,14,None,YES,numeric,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
492,eicu,eicu_crd,pivoted_vital_other,patientunitstayid,1,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
493,eicu,eicu_crd,pivoted_vital_other,chartoffset,2,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
494,eicu,eicu_crd,pivoted_vital_other,entryoffset,3,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES


### Define  important tables and columns

- some tables are more relevant than others
- also add completeness next to each column

In [113]:
tables_to_keep = [
    'admissiondx', 
    'apacheapsvar',
    'apachepatientresult', 
    'apachepredvar', 
    'diagnosis', 
    'hospital',
    'lab', 
    'medication', 
    'nurseassessment', 
    'nursecare', 
    'nursecharting',
    'pasthistory', 
    'patient', 
    'treatment',
    'vitalaperiodic',
    'vitalperiodic',
    'pivoted_bg',
    'pivoted_infusion',
    'pivoted_lab',
    'pivoted_med',
    'pivoted_o2',
    'pivoted_score',
    'pivoted_uo',
    'pivoted_vital',
    'pivoted_vital_other',
]

In [119]:
# restrict to tables that are useful for modelling, keep relevant cols
df_schema_final = df_schemas[df_schemas.table_name.isin(tables_to_keep)]
df_schema_final = df_schema_final.set_index('table_name')
df_schema_final = df_schema_final[
    [
        'column_name',
        'ordinal_position',
        'data_type',
        'udt_name'       
    ]
]

In [73]:
# calculate completeness info for each col
completeness = []
for table in df_schema_final.index.unique():
    table_cols = df_schemas.loc[table].column_name.values
    print(table, len(table_cols))
    completeness.append(get_column_completeness(table, table_cols, conn))

admissiondx 6
apacheapsvar 26
apachepatientresult 23
apachepredvar 51
diagnosis 7
hospital 4
lab 10
medication 15
nurseassessment 8
nursecare 8
nursecharting 8
pasthistory 8
patient 29
treatment 5
vitalaperiodic 13
vitalperiodic 19
pivoted_bg 10
pivoted_infusion 10
pivoted_lab 24
pivoted_med 13
pivoted_o2 6
pivoted_uo 4
pivoted_vital 14
pivoted_vital_other 18


In [124]:
# add completeness info and save modelling_schema table
df_schema_final.insert(len(df_schema_final.columns), 'completeness', pd.concat(completeness).values/100.)
df_schema_final.to_csv('../setup/modelling_tables_schema.csv')

### Modify the `modelling_tables_schema.csv` manually

I went through all the columns to:
- identify the time domain column
- mark certain cols for deletion which were either non-relevant or not populated (relying on the `get_column_completeness`)
- using this updated version, we now can create a naming object that holds all relevant table info for cohort building.
- note this final updated version (`modelling_schema.csv`) was moved to the etl folder

#### Have a look at the final `modelling_schema.csv`

In [125]:
df_schema_final = load_schema_for_modelling()
df_schema_final.head(20)

,column_name,ordinal_position,data_type,udt_name,time_var,numeric,to_keep,data_table,completeness
table_name,,,,,,,,,
admissiondx,admissiondxid,1,integer,int4,0,1,1,1,1.0
admissiondx,patientunitstayid,2,integer,int4,0,1,1,1,1.0
admissiondx,admitdxenteredoffset,3,integer,int4,1,1,1,1,1.0
admissiondx,admitdxpath,4,character varying,varchar,0,0,1,1,1.0
admissiondx,admitdxname,5,character varying,varchar,0,0,0,1,1.0
admissiondx,admitdxtext,6,character varying,varchar,0,0,0,1,1.0
apacheapsvar,apacheapsvarid,1,integer,int4,0,1,1,1,1.0
apacheapsvar,patientunitstayid,2,integer,int4,0,1,1,1,1.0
apacheapsvar,intubated,3,smallint,int2,0,1,1,1,1.0
